## Starwar API

In [ ]:
#r "nuget: GraphQL.Client, 4.0.2"
#r "nuget: GraphQL.Client.Serializer.Newtonsoft, 4.0.2"


Installed Packages GraphQL.Client, 4.0.2 GraphQL.Client.Serializer.Newtonsoft, 4.0.2

In [ ]:
using System.Collections.Generic;


public class PersonAndFilmsResponse
{
    public PersonContent Person { get; set; }

    public class PersonContent
    {
        public string Name { get; set; }

        public FilmConnectionContent FilmConnection { get; set; }

        public class FilmConnectionContent
        {
            public List<FilmContent> Films { get; set; }

            public class FilmContent
            {
                public string Title { get; set; }
            }
        }
    }
}


In [ ]:
using System;
using System.Net.Http;
using System.Linq;
using System.Text.Json;
using System.Threading.Tasks;
using GraphQL;
using GraphQL.Client.Http;
using GraphQL.Client.Serializer.Newtonsoft;

// var EndPoint = "https://swapi.apis.guru/";
var httpClientHandler = new HttpClientHandler();
httpClientHandler.ServerCertificateCustomValidationCallback = (message, cert, chain, sslPolicyErrors) =>
{
    return true;
};
// var httpClient = new HttpClient(httpClientHandler) { BaseAddress = new Uri(EndPoint) };

var ClientOpts = new GraphQLHttpClientOptions{HttpMessageHandler = httpClientHandler, EndPoint = new Uri("https://swapi.apis.guru/") };

var graphQLClient = new GraphQLHttpClient(ClientOpts, new NewtonsoftJsonSerializer());

// graphQLClient.HttpClient.

var personAndFilmsRequest = new GraphQLRequest
{
    Query = @"
    query PersonAndFilms($id: ID) {
        person(id: $id) {
            name
            filmConnection {
                films {
                    title
                }
            }
        }
    }",
    OperationName = "PersonAndFilms",
    Variables = new
    {
        id = "cGVvcGxlOjE="
    }
};

var graphQLResponse = await graphQLClient.SendQueryAsync<PersonAndFilmsResponse>(personAndFilmsRequest);
// display("raw response:");
// display(JsonSerializer.Serialize(graphQLResponse, new JsonSerializerOptions { WriteIndented = true }));
Console.WriteLine("raw response:");
Console.WriteLine(JsonSerializer.Serialize(graphQLResponse, new JsonSerializerOptions { WriteIndented = true }));

// Console.WriteLine();
Console.WriteLine($"Name: {graphQLResponse.Data.Person.Name}");
var films = string.Join(", ", graphQLResponse.Data.Person.FilmConnection.Films.Select(f => f.Title));
Console.WriteLine($"Films: {films}");

// Console.WriteLine();
// Console.WriteLine("Wait for completion ...");
// Console.ReadKey();


raw response:
{
  "Data": {
    "Person": {
      "Name": "Luke Skywalker",
      "FilmConnection": {
        "Films": [
          {
            "Title": "A New Hope"
          },
          {
            "Title": "The Empire Strikes Back"
          },
          {
            "Title": "Return of the Jedi"
          },
          {
            "Title": "Revenge of the Sith"
          }
        ]
      }
    }
  },
  "Errors": null,
  "Extensions": null
}
Name: Luke Skywalker
Films: A New Hope, The Empire Strikes Back, Return of the Jedi, Revenge of the Sith


In [ ]:
$payload = @{
    query = 'query PersonAndFilms($id: ID) { person(id: $id) { name filmConnection { films { title } } } }';
    OperationName = "PersonAndFilms";
    variables = 
    @{
        id = "cGVvcGxlOjE="
    }
}

In [ ]:
$body = $payload |ConvertTo-Json -Depth 5 -Compress
$body

{"query":"query PersonAndFilms($id: ID) { person(id: $id) { name filmConnection { films { title } } } }","OperationName":"PersonAndFilms","variables":{"id":"cGVvcGxlOjE="}}


In [ ]:
curl.exe -k -X POST -H "Content-Type:application/json" https://swapi.apis.guru/ -d ($body -replace '"', '\"')
# curl.exe -k -X POST -H "Content-Type:application/json" https://swapi.apis.guru/ -d -d $body

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
{"data":{"person":{"name":"Luke Skywalker","filmConnection":{"films":[{"title":"A New Hope"},{"title":"The Empire Strikes Back"},{"title":"Return of the Jedi"},{"title":"Revenge of the Sith"}]}}}}
100   368  100   196  100   172   1882   1652 --:--:-- --:--:-- --:--:--  3572


In [ ]:
$xbody = [System.Text.Encoding]::UTF8.GetBytes($body)
$ret = Invoke-RestMethod -Uri https://swapi.apis.guru/ -SkipCertificateCheck -Body $xbody -ContentType "application/json"

## Github GraphQL

In [ ]:
. D:\projects\binder\bruno-notebooks\.env\github.ps1

$headers = @{Authorization = ("bearer {0}" -f  $github_token); "Content-Type" = "application/json"; Accept="*/*"}

In [ ]:
$q = 'query orginfo($mylogin: String!) {
    organization(login: $mylogin) {
      repositories(first: 10, orderBy: {field: STARGAZERS, direction: DESC}) {
        nodes {
          nom: name
        }
      }
    }
  }'

$q2 = 'query {
  organization(login: "jscomplete") {
    repositories(first: 10, orderBy: {field: STARGAZERS, direction: DESC}) {
      nodes {
        nom: name
      }
    }
  }
}'
$payload = @{
    query = $q
    variables = @{mylogin = 'jscomplete'}
}

In [ ]:
# $xbody = [System.Text.Encoding]::UTF8.GetBytes((($payload |ConvertTo-Json) -replace '\r', '') )
$xbody = (($payload |ConvertTo-Json) -replace '\r', '')
# $ret2 = Invoke-RestMethod -Uri https://api.github.com/graphql -Headers $headers -Body $xbody -ContentType 'application/json' -Method Post
$ret2 = Invoke-RestMethod -Uri https://api.github.com/graphql -Headers $headers -Body $xbody -ContentType 'application/json' -Method Post

In [ ]:
($payload |ConvertTo-Json -Compress) -replace '\\r\\n', ''

{"query":"query {  organization(login: \"jscomplete\") {    repositories(first: 10, orderBy: {field: STARGAZERS, direction: DESC}) {      nodes {        nom: name      }    }  }}"}


In [ ]:
# $ret2 |ConvertTo-Json -Depth 25 | Out-File -FilePath D:\projects\binder\bruno-notebooks\github_schema.json -Encoding utf8NoBOM -Force
# $ret2|ConvertTo-Json
$ret2.data.organization.repositories.nodes
# $headers
# ($payload |ConvertTo-Json) -replace '\\r', ''

# $payload
# curl.exe -H "Authorization: bearer $github_token" -X POST -d `
# ($payload |ConvertTo-Json -Compress) `
# https://api.github.com/graphql



nom
---
learn-fullstack-javascript
ngs
advanced-nodejs
graphql-in-action
book-node-beyond-basics
advanced-react
graphfront
reactful
rgs-star-match
graphfront-ui

